In [ ]:
import googleapiclient.discovery
import pandas as pd
import time

api_service_name = "youtube"
api_version = "v3"
DEVELOPER_KEY = 'API_KEY_USED'  # Replace with your actual API key

youtube = googleapiclient.discovery.build(
    api_service_name, api_version, developerKey=DEVELOPER_KEY)

def get_comments(video_id):
    comments = []
    nextPageToken = None

    while True:
        try:
            request = youtube.commentThreads().list(
                part="snippet",
                videoId=video_id,
                maxResults=100,
                pageToken=nextPageToken if nextPageToken else ''
            )

            response = request.execute()

            for item in response['items']:
                comment = item['snippet']['topLevelComment']['snippet']
                public = item['snippet']['isPublic']
                comments.append([
                    comment['authorDisplayName'],
                    comment['publishedAt'],
                    comment['likeCount'],
                    comment['textOriginal'],
                    comment['videoId'],
                    public
                ])

            nextPageToken = response.get('nextPageToken')
            if not nextPageToken:
                break

            time.sleep(1)  # Add a delay to respect API rate limits

        except Exception as e:
            print(f"Error fetching comments for video ID {video_id}: {e}")
            break

    df = pd.DataFrame(comments, columns=['author', 'updated_at', 'like_count', 'text', 'video_id', 'public'])
    return df

df = pd.DataFrame()
videos = ['cBpGq-vDr2Y', 'AzALiMv6hpY', 'oqjWnA7J0f0', 'RBe0MVJ_XlU', 'OMUSjIUOKiM', 'ZdjE0i8jozo', 'To54FvXT5m0', 'JwAfHEHQKto', '7Dcv9lyikrw', 'YmwskGLycHo', 'EeTs67l69B4', 'WzPjIIjfnso', 'Ipp4NqtL12U', 'SNqF6yMmCPc', '_3_ibMon2nU', 'NwR-jbIIILc', 'P6Crk0jWxk4', 'k5P4eLo2SRI', 'JwEYDgCsQrg', 'v2KhCGI1dUw', 'vozVLKLCo_k']

for video_id in videos:
    df2 = get_comments(video_id)
    df = pd.concat([df, df2], ignore_index=True)

print(df.head())


                author            updated_at  like_count  \
0        @abdlkrim9296  2024-07-15T12:07:27Z           0   
1       @JasperStaal86  2024-07-14T17:24:11Z           0   
2       @oneegiiie1490  2024-07-14T06:09:54Z           0   
3  @hariyanhariyan7556  2024-07-14T04:35:29Z           0   
4          @SoulAmatus  2024-07-13T23:15:56Z           0   

                                                text     video_id  public  
0  i'm not here for apple or review, i'm here for...  cBpGq-vDr2Y    True  
1           What is that fancy weather app at 9:56 👀  cBpGq-vDr2Y    True  
2                                         4:56 ✈️ 🏙️  cBpGq-vDr2Y    True  
3                                          MrBeast ⚡  cBpGq-vDr2Y    True  
4  I have a 7 plus and want to upgrade. I’m hopin...  cBpGq-vDr2Y    True  


In [ ]:
df.shape

(61306, 6)

In [ ]:
df.to_csv('comments.csv')